In [ ]:
import pandas as pd
import os
import seaborn as sns

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime import datetime

# ? Se parametrizan las variables a usar en la función de conexión (connect)
host = 'bcgs-multiproposito-rds-alfa-replica.c108sqqsuwz4.us-east-1.rds.amazonaws.com'
database = 'bcgs'
user = 'vigente'
password = 'tDL!cg!ZhKRRE2iC9U6Z2#nY'
port = '5432'
schema = 'bpmcat'

consulta_1 = """ 
    select t1.municipio
        ,t1.npn
        ,t1.nro_ficha
        ,t1.radicado
        ,(case when t1.vr_nombre_proceso like 'Comple%%' then 'Actualización'
            when t1.vr_nombre_proceso like 'Forma%%' then 'Actualización'
            when t1.vr_nombre_proceso like 'Muta%%' then 'Conservación'				
            when t1.vr_nombre_proceso like 'Recti%%' then 'Conservación'
            else t1.vr_nombre_proceso
            end) "Proceso"
        ,t1.vr_nombre_proceso	
        ,t1.estado_tramite
        ,t1.actividad
        ,t1.vr_estado_paso
        ,t2.vr_adt_usrins
        ,cast(t1.fecha_radicado as date) fecha_radicado
        ,t1.usuario
        ,cast(t2.dt_adt_fchupd as date) fecha_actualizacion_tramite
        ,t4.vr_task_nom nombre_tarea
        ,t4.vr_task_con 
        ,t4.vr_task_est tarea_estado
        ,(cast(t2.dt_adt_fchupd as date) - cast(t1.fecha_radicado as date)) vida_tramite
        ,(cast(now() as date) - cast(t1.fecha_radicado as date)) vigente_tramite_desde
    from vw_rpt_tramites_en_proceso t1 
    inner join tb_bpm_egn_proegn t2 on t1.radicado = t2.vr_proegn_rad
    inner join tb_bpm_mdl_proces t3 on t2.nb_proces_idn = t3.nb_proces_idn
    inner join tb_bpm_mdl_task t4 on t3.nb_task_idn = t4.nb_task_idn;
"""

try:
    print("Inicio de la conexión")
    
    # ? Se asigna a una variable los resultados de leer una consulta SQL con una conexión previamente generada
    # ? Parámetros de conexión desde sqlalchemy
    url = f"postgresql://{user}:{password}@{host}:{port}/{database}"
    engine = create_engine(url, pool_size=50, echo=False)

    # ? Asignnación de una conexión a una variable
    conexion_sqlalq = engine.connect()

    # Establecer el esquema en la conexión
    conexion_sqlalq.execute(text(f'SET search_path TO {schema}'))

    # TODO: Indicador 1
    # ? Generación de Dataframe por medio de consulta en sql. Función read_sql_query.
    consulta_sql_1 = pd.read_sql_query(consulta_1, conexion_sqlalq)

    # TODO: Indicador 1
    # ? Se construye el dataframe a partir de la lectura de la consulta (usando la función read_sql_query)
    df_consulta_indicador_1 = pd.DataFrame(consulta_sql_1)
    
    print("Conexión exitosa ...")
    
    # Cierre de la conexión
    conexion_sqlalq.close()

except Exception as e:
    print(f"Error durante la conexión o ejecución de la consulta: {e}")
    consulta_sql_1 = None

# Verificar si consulta_sql_3 se definió correctamente
if consulta_sql_1 is not None:
    print(consulta_sql_1)
else:
    print("La consulta no se pudo ejecutar y la variable consulta_sql_3 no se definió.")

fecha_actual = datetime.now()
fecha_archivo = str(fecha_actual.strftime("%Y-%m-%d"))

query_actualizacion_estado = "Proceso == 'Actualización' and vr_estado_paso == 'ATENDIENDO'"
query_conservacion_estado = "Proceso == 'Conservación' and vr_estado_paso == 'ATENDIENDO'"

# TODO: Total por estado y proceso
df_consulta_indicador_1_actualizacion_estado = df_consulta_indicador_1.query(query_actualizacion_estado)
df_consulta_indicador_1_actualizacion_estado_proceso = pd.DataFrame(df_consulta_indicador_1_actualizacion_estado['vr_nombre_proceso'].value_counts()).rename(columns={'count':'total_procesos'}).reset_index().rename(columns={'vr_nombre_proceso':'nombre_proceso'})

df_consulta_indicador_1_actualizacion_estado_responsable = pd.DataFrame(df_consulta_indicador_1_actualizacion_estado['usuario'].value_counts()).rename(columns={'count':fecha_archivo}).reset_index()


In [2]:
RUTA_DIRECTORIO_TRAMITE_RESPONSABLE = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuirica\1.Indicadores_Estrategicos\1.1.Responsable_Por_Tramite_Activo"
NOMBRE_XLSX_TRAMITE_RESPONSABLE = 'Responsable_Tramite_En_Atencion.xlsx'

ruta_tramite_responsable = os.path.join(RUTA_DIRECTORIO_TRAMITE_RESPONSABLE, NOMBRE_XLSX_TRAMITE_RESPONSABLE)

df_consulta_indicador_1_actualizacion_estado_responsable.to_excel(ruta_tramite_responsable, sheet_name='Rendimientos')
print(f"Se crea XLSX {NOMBRE_XLSX_TRAMITE_RESPONSABLE}")

,usuario,2024-05-22
0,melany.delahoz,19
1,admin,17
2,hernando.sarmiento,10
3,julie.castano,10
4,kevin.castro,5
5,rafael.molina,5
6,jair.benedetty,5
7,harlingto.padilla,4
8,maria.hernandez,4
9,luis.jimenez,4
